In [1]:
%cd d:\Laptop_Projects\LLMEval-1

d:\Laptop_Projects\LLMEval-1


In [2]:
from collections import Counter
import numpy as np
from summac.model_guardrails import NERInaccuracyPenalty
import nltk
import spacy
import json
import pandas as pd
import string
import tfidf_matcher as tm
from fuzzywuzzy import fuzz
from server.metrics.faithfulness_helper import FaithfulnessHelper,get_quartiles_faithfulness
from server.metrics.stylistic import FaithfulnessEvaluator

data = json.load(open(r'preprocess\experiments\data\pairwise_evaluation_w_embeddings.json'))

In [3]:
def save_json(data, filepath=r'new_data.json'):
    with open(filepath, 'w') as fp:
        json.dump(data, fp, indent=4)

Load data, all class objects and meta data

In [4]:
data = json.load(open(r'D:\Laptop_Projects\LLMEval-1\preprocess\experiments\data\Faithfulness_test.json'))
df = pd.DataFrame(data)
fh = FaithfulnessHelper()
ranges = get_quartiles_faithfulness(df)
fe = FaithfulnessEvaluator(ranges) #Pass empty string if you dont have Faithfulness_test.json

New Instance example

In [5]:
article_text = 'Polls close at 11 p.m. Eastern time. In the 2022 state primaries, first results were reported 11 minutes later, and the last update of the night was at 6:10 a.m. Eastern time with 48 percent of votes reported.All active registered voters were mailed a ballot, which must be postmarked by Election Day and received by March 12. Early in-person voting was also available.California is one of five states on Tuesday holding primaries for the presidential race as well as its state primary. In state and congressional primaries, candidates from all parties are listed on one ballot, and the top two vote-getters advance to the general election. In the presidential primary, Republican voters were required to be registered with their party to vote for its nominee, but Democrats and unaffiliated voters could participate in the Democratic election.'
summ ='In the 2022 California state primaries, polls closed at 11 p.m. Eastern time. First results were reported 11 minutes later, and the last update of the night was at 6:10 a.m. Eastern time with 48 percent of votes reported. California held primaries for both the presidential race and its state primary, using a top-two vote-getter system for state and congressional primaries.'
fe.default(article_text,summ)

(0.6666666666666666, 'good')

Calculate scores for whole dataset

In [6]:
article_text = []
LLM_text = []
for i,datum in enumerate(data):
    print('{}/{}'.format(i, len(data)))
    article=datum['article_text']
    LLM=datum['text-davinci-002_summary']
    datum['faithfullness'], datum['faithfullness_binary'] = fe.default(article,LLM) # Remove 'datum['faithfullness_binary']' if you dont have Faithfulness_test.json 
    print("score",datum['faithfullness'])
    # del datum['full_embedding']
    # del datum['writer_summary_embedding']
    # del datum['llm_summary_embedding']

0/599
score 0.5
1/599
score 0.8
2/599
score 0.7777777777777778
3/599
score 0.6666666666666666
4/599
score 0.2
5/599
score 0.75
6/599
score 0.6
7/599
score 0.0
8/599
score 0.75
9/599
score 0.5714285714285714
10/599
score 0.7777777777777778
11/599
score 0.8
12/599
score 0.0
13/599
score 0.6666666666666666
14/599
score 0.0
15/599


In [7]:
save_json(data,r'D:\Laptop_Projects\LLMEval-1\preprocess\experiments\data\Faithfulness_binary.json')

Testing code to view quartiles and counts of labels

In [8]:
data_faith = json.load(open(r'D:\Laptop_Projects\LLMEval-1\preprocess\experiments\data\Faithfulness_test.json'))
df = pd.DataFrame(data)
df.to_csv('faithfulness.csv', index = True) 

View the quartiles

In [9]:
Q1 = np.percentile(df['faithfullness'], 25)
Q2 = np.percentile(df['faithfullness'], 50)
Q3 = np.percentile(df['faithfullness'], 75)
print("Q1", Q1)
print("Q2", Q2)
print("Q3", Q3)
# define ranges, add another range for all quartiles.
range1 = (df['faithfullness'].min(), Q1)
range2 = (Q1,Q2)
range3 = (Q2,Q3)
range4 = (Q3, df['faithfullness'].max())

print(f"Range 1: {range1}")
print(f"Range 2: {range2}")
print(f"Range 3: {range3}")
print(f"Range 4: {range4}")

Q1 0.2
Q2 0.4
Q3 0.6
Range 1: (0.0, 0.2)
Range 2: (0.2, 0.4)
Range 3: (0.4, 0.6)
Range 4: (0.6, 1.0)


Counts of each label

In [10]:
for ind in df.index:
    if df['faithfullness'][ind] <= Q1:
        df['faithfullness'][ind]= 'bad'
    elif Q1 < df['faithfullness'][ind] < Q2:
        df['faithfullness'][ind]= 'low'
    elif Q2 <= df['faithfullness'][ind] < Q3:
        df['faithfullness'][ind]= 'avg'
    else:
        df['faithfullness'][ind]= 'good'
df['faithfullness'].value_counts()

C:\Users\aryam\AppData\Local\Temp\ipykernel_13180\1853716175.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['faithfullness'][ind]= 'avg'


bad     169
avg     162
good    155
low     113
Name: faithfullness, dtype: int64